In [ ]:
from random import choice, random, seed
# seed(1211)  # arbitrary seed for testing


class Board:
    def __init__(self, size=4, start=(0, 0), pit_rate=0.1, 
                 gold_loc=None, wumpus_loc=None, n_wumpus=1):
        """
        The board is represented as a list of lists.
        Most placings are either random or intended to be specified by the user.
        I wanted to also handle a non-square board, but did not get to implementing it.
        :param size: board size NXN matrix, default is 4.
        :param start: starting location. Default is 0,0, which is top left.
        :param pit_rate: likelihood of an unoccupied space to become a pit.
        :param gold_loc: the gold's starting location. Random if None.
        :param wumpus_loc: the Wumpus starting location. Random if None.
                            The Wumpus doesn't move in this implementation :(
        :param n_wumpus: theoretically, could have more than one wumpus.
        """
        self.size, self.start, self.agent_location, self.pit_rate, self.gold_loc,\
        self.wumpus_loc, self.n_wumpus = size, start, start, pit_rate,\
                                         gold_loc, wumpus_loc, n_wumpus
        self.board = self.generate_board(size, start, gold_loc)
        self.pit_locations = []
        self.generate_pits(pit_rate)
        self.add_wumpus(wumpus_loc)
        self.breezes = {}
        self.stenches = {}
        self.breezes_and_stench(self.pit_locations, self.wumpus_loc)
        return

    def generate_board(self, size, start, gold_loc):
        """ :returns a list of lists of size n
                empty locations are 0s, agent is A, and gold is g"""

        # create nXn matrix
        board = [[0 for i in range(size)] for j in range(size)]

        # gold appears at random location, or at a specified one.
        # this should handle errors, but for now left to rely on the user.
        # theoretically the gold could be in the starting square,
        # but for ease of implementation, it isn't
        while gold_loc is None:
            x, y = choice(range(size)), choice(range(size))
            if [x, y] != start:
                gold_loc = (x, y)
        board[gold_loc[0]][gold_loc[1]] = "g"
        self.gold_loc = gold_loc

        # place the agent at the chosen start location
        board[start[0]][start[1]] = "A"

        return board

    def generate_pits(self, pit_rate):
        # for each unoccupied square,
        # generate random floating point in range 0-1
        # if it is smaller than the pit-rate, the room becomes a pit.
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0 and random() <= pit_rate:
                    self.board[i][j] = "p"
                    self.pit_locations.append((i, j))
        return

    def add_wumpus(self, wumpus_loc):
        # randomize an unoccupied square, place wumpus,
        # unless a location is specified
        # not currently implementing multiple wumpusi
        while wumpus_loc is None:
            x, y = choice(range(self.size)), choice(range(self.size))
            if self.board[x][y] == 0:
                wumpus_loc = (x, y)
        self.board[wumpus_loc[0]][wumpus_loc[1]] = "w"
        self.wumpus_loc = wumpus_loc
        return

    def breezes_and_stench(self, pit_locations, wumpus_loc):
        # will be used for perceiving stenches and breezes
        for pit in pit_locations:
            for room in self.adjacent(pit):
                self.breezes[room] = True
        for room in self.adjacent(wumpus_loc):
            self.stenches[room] = True
        return

    @staticmethod
    def adjacent(loc):
        """
        checks for adjacency of rooms.
        Does not care for the borders of the board, though.
        Decided to handle that in each call,
        although would make sense to do here instead.
        :param loc: the room to check for adjacency.
        :return: all adjacent rooms as a list of tuples [(X,Y),...]
        """
        return [(loc[0] + 1, loc[1]), (loc[0] - 1, loc[1]), 
                (loc[0], loc[1] + 1), (loc[0], loc[1] - 1)]



In [ ]:
import itertools
from heapq import heappop, heappush


class PriorityQueue:
    def __init__(self):
        """
        Similar to the usual priority queue, but using
        (priority, -count) when pushing an entry, so newer
        entries are popped first.
        """
        self.pq = []  # list of entries arranged in a heap
        self.entry_finder = {}  # mapping of tasks to entries
        self.REMOVED = '<removed-task>'  # placeholder for a removed task
        self.counter = itertools.count()  # unique sequence count

    def push(self, task, priority=0):
        'Add a new task or update the priority of an existing task'
        if task in self.entry_finder:
            self.remove(task)
        count = next(self.counter)
        # using -count so we have LIFO instead of FIFO
        entry = [(priority, -count), count, task]
        self.entry_finder[task] = entry
        heappush(self.pq, entry)

    def remove(self, task):
        'Mark an existing task as REMOVED.  Raise KeyError if not found.'
        entry = self.entry_finder.pop(task)
        entry[-1] = self.REMOVED

    def pop(self):
        'Remove and return the lowest priority task. Raise KeyError if empty.'
        while self.pq:
            priority, count, task = heappop(self.pq)
            if task is not self.REMOVED:
                del self.entry_finder[task]
                return task
        raise KeyError('pop from an empty priority queue')

    def __len__(self):
        return len(self.entry_finder)

In [ ]:
# from PriorityQueueLIFO import PriorityQueue

class kb:
    def __init__(self, board):
        """
        a class for keeping the information that is known to the agent
        separate from the full state of the board.
        The KB updates based on agent actions, and is used to
        confirm different results with the board.
        :param board: a board object.
        """
        self.board = board
        self.breezes, self.stenches, self.glitter = {}, {}, {}
        self.explored, self.safe_rooms, room_risks = {}, {}, {}

        for row in range(board.size):
            for column in range(board.size):
                # assign no-risk to rooms as a starting point
                # risk will be a way to evaluate pits and wumpus
                # actually inferring a pit is really difficult,
                # even if we know how many there are, which we don't.
                # see notes.
                room_risks[(row, column)] = 0

        self.agent_location = board.agent_location

        self.room_risks = room_risks
        self.explored[board.start] = True
        self.frontier = PriorityQueue()
        return

    def increase_risk_adjacent(self, loc):
        for room in self.adjacent(loc):
            self.room_risks[room] += 1
        return


    @staticmethod
    def adjacent(loc):
        return [(loc[0] + 1, loc[1]), (loc[0] - 1, loc[1]), 
                (loc[0], loc[1] + 1), (loc[0], loc[1] - 1)]



In [ ]:
# from WumpusBoard import Board
# from WumpusKB import kb


class agent:
    def __init__(self, kb):
        """
        a class for the agent to perform actions.
        Relies on the KB and the board.
        :param kb: a kb object
        """
        self.kb = kb
        return

    def perceive(self, location):
        no_stench = False
        no_breeze = False

        # check for game-over
        try:
            if (location in self.kb.board.pit_locations)\
                    or (location == self.kb.board.wumpus_loc):
                return "GAME OVER"
        except KeyError:
            pass

        # check for stench
        try:
            if self.kb.board.stenches[location]:
                self.kb.breezes[location] = True
                self.kb.increase_risk_adjacent(location)
        except KeyError:
            no_stench = True

        # check for breeze
        try:
            if self.kb.board.breezes[location]:
                self.kb.breezes[location] = True
                self.kb.increase_risk_adjacent(location)
        except KeyError:
            no_breeze = True

        # if neither stench nor breeze, make adjacent squares "safe"
        if no_breeze and no_stench:
            for room in kb.adjacent(location):
                if room not in kb.explored:
                    if (0 <= room[0] <= self.kb.board.size)
                    and (0 <= room[1] <= self.kb.board.size):
                        kb.frontier.push(room, 0) 
                        # TODO: safe rooms should always be selected for exploration
                        kb.safe_rooms[room] = True

        # check for glitter
        try:
            if self.kb.board.gold_loc == location:
                self.kb.glitter[location] = True
        except KeyError:
            pass

        return

    # adds the adjacent locations to the frontier
    def add_to_frontier(self, loc):
        for room in kb.adjacent(loc):
            if room not in kb.explored:
                # only adds rooms that have not been explored,
                # and inside the board
                if (0 <= room[0] <= self.kb.board.size)
                and (0 <= room[1] <= self.kb.board.size):
                    self.kb.frontier.push(room, self.kb.room_risks[room])
                    # since frontier is a priority que,
                    # the order to explore will be by risk level
                    # need to adjust so we explore from the last added,
                    #not the first
        return

    def find_route(self, target_room, allowed_rooms):
        # this would have used the A* search
        """
        finds the shortest path to a room using A* search
        only explores within the list of visited rooms
        :param target_room: room as (X, Y)
        :param allowed_rooms: list of rooms (X, Y)
        :return: ordered list of rooms to pass.
                    allowed rooms are either
                    rooms we visited or deemed safe
        """
        return

    def move(self):
        """
        the 'move' function was intended to move the agent
        to the safest nearest space, through the shortest route.
        If the gold was found, it would either grab it and move
        to the exit, or move to the gold's room and then grab
        it and move back to the exist.
        :return:
        """
        try:
            if kb.glitter != {}:
                # do we have gold? go to exit.
                # no gold? are we at the gold location? grab, go to exit
                # not at gold location? go to gold location.
                return

        except:
            target_room = self.kb.frontier.pop(0)

            # move to least risky square through
            # shortest path of explored rooms
            #    in case of tie on risk: randomize
            #    check for precepts in new location,
            #    update the exploration frontier
            # self.perceive(loc)
            # self.add_to_frontier(loc)
            return


board = Board()
kb = kb(board)
agent = agent(kb)